## If you want to use the Selenium code

Install selenium package with your command prompt.

install the chrome driver for your google chrome version. (you must have google chrome browser)

change the url link on the code ( I typed 'crimes in south africa' on the google search bar, and copied the search link)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time

In [ ]:
PATH = "C:\\Users\\Tim\\Desktop\\Web Scraping Images\\chromedriver.exe"

In [ ]:
wd = webdriver.Chrome(PATH)

In [ ]:
def get_images_from_google(wd, delay, max_images):
	def scroll_down(wd):
		wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(delay)

    # first go to google and type in your search phrase, after you click on search, copy the link,
    # and paste it here
	url = "https://www.google.com/search?q=crimes+in+south+africa&client=opera&hs=1mV&sxsrf=ALiCzsZ-O4kjhnK7VxUj5hFbp9PPOLLY-Q:1669355851460&source=lnms&tbm=isch&sa=X&ved=2ahUKEwiaoZ6U08j7AhUqTaQEHbaBDjEQ_AUoAXoECAEQAw&biw=1600&bih=767&dpr=1"
	wd.get(url)

	image_urls = set()
	skips = 0

	while len(image_urls) + skips < max_images:
		scroll_down(wd)
        #right click on the google page, click on inspect element and get the 
		thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

		for img in thumbnails[len(image_urls) + skips:max_images]:
			try:
				img.click()
				time.sleep(delay)
			except:
				continue

			images = wd.find_elements(By.CLASS_NAME, "Q4LuWd")
			for image in images:
				if image.get_attribute('src') in image_urls:
					max_images += 1
					skips += 1
					break

				if image.get_attribute('src') and 'http' in image.get_attribute('src'):
					image_urls.add(image.get_attribute('src'))
					print(f"Found {len(image_urls)}")

	return image_urls

In [ ]:
def download_image(download_path, url, file_name):
	try:
		image_content = requests.get(url).content
		image_file = io.BytesIO(image_content)
		image = Image.open(image_file)
		file_path = download_path + file_name

		with open(file_path, "wb") as f:
			image.save(f, "JPEG")

		print("Success")
	except Exception as e:
		print('FAILED -', e)

In [ ]:
urls = get_images_from_google(wd, 1, 2)
cwd = os.getcwd()
IMAGE_FOLDER = 'download'
os.makedirs(
    name=f'{cwd}/{IMAGE_FOLDER}',
    exist_ok=True
)
for i, url in enumerate(urls):
	download_image('download/', url, str(i) + ".jpg")

wd.quit()